# CoGames: A Game Environment for the Alignment League Benchmark

<p align="center">
  <a href="https://pypi.org/project/cogames/">
    <img src="https://img.shields.io/pypi/v/cogames" alt="PyPi version">
  </a>
  <a href="https://pypi.org/project/cogames/">
    <img src="https://img.shields.io/pypi/pyversions/cogames" alt="Python version">
  </a>
  <a href="https://discord.gg/secret-hologenesis">
    <img src="https://img.shields.io/discord/1309708848730345493?logo=discord&logoColor=white&label=Discord" alt="Discord">
  </a>
  <a href="https://deepwiki.com/Metta-AI/cogames">
    <img src="https://deepwiki.com/badge.svg" alt="Ask DeepWiki">
  </a>
  <a href="<<colab-link>>">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab">
  </a>

  <a href="https://softmax.com/">
    <img src="https://img.shields.io/badge/Softmax-Website-849EBE?logo=" alt="Softmax website">
  </a>
</p>

The [Alignment League Benchmark (ALB)](https://www.softmax.com/alignmentleague) is a suite of multi-agent games, designed to measure how well AI agents align, coordinate, and collaborate with others (both AIs and humans).

CoGames is the games environment for ALB. You can use it to:

* create new games
* train agents to play existing ALB games
* submit those agents to the ALB leaderboard

There's one ALB game right now: Cogs vs Clips.

# Quick Start

## Step 1: Install CoGames

Install [cogames](https://pypi.org/project/cogames/) as a Python package.
```bash
pip install cogames
```

<details><summary>Using uv</summary>

```bash
# Install uv
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create a virtual environment
uv venv .venv
source .venv/bin/activate

# Install cogames
uv pip install cogames
```

</details>
<details><summary>Using Docker</summary>

```dockerfile
# Ensure Python 3.12 is available
FROM python:3.12-slim

# Ensure C/C++ compiler is available
RUN apt-get update && \
    apt-get install -y --no-install-recommends build-essential && \
  rm -rf /var/lib/apt/lists/*

# Install cogames
RUN pip install --no-cache-dir cogames
```

</details>
<details><summary>Using Colab</summary>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](<<colab-link>>)

</details>

## Step 2: Play Cogs vs Clips

Play an easy mission in Cogs vs Clips using:
```bash
cogames tutorial play
```
The game will open in a new window, and the terminal will give you instructions to complete training mission.

## Step 3: Submit a policy to the leaderboard

1. Log into the ALB leaderboard with your GitHub account.

    ```bash
    cogames login
    ```

2. Upload one of our starter policies under your name.

    ```bash
    cogames upload --policy "class=cogames.policy.starter_agent.StarterPolicy" --name "$USER.README-quickstart-starter-policy" --skip-validation
    ```

3. List the seasons currently running.

    ```bash
    cogames seasons
    ```

4. Submit the policy to one of the running seasons.

    ```bash
    cogames submit "$USER.README-quickstart-starter-policy:v1" --season beta-teams-small
    ```

# Tutorials

To learn more, see:

1. [Creating a policy](tutorials/01_MAKE_POLICY.md): Creating a custom policy and evaluating it
2. [Training](tutorials/02_TRAIN.md): Training a custom policy and evaluating it
3. [Submitting](tutorials/03_SUBMIT.md): Submitting to the leaderboard and understanding the results

If you want help, or to share your experience, join [the Discord](https://discord.gg/secret-hologenesis).

# About the game

CogsGuard is a cooperative territory-control game. Teams of AI agents ("Cogs") work together to capture and defend
junctions against automated opponents ("Clips") by:

* gathering resources and depositing them at controlled junctions
* acquiring specialized roles (Miner, Aligner, Scrambler, Scout) at gear stations
* capturing neutral junctions using Aligners (costs 1 heart)
* disrupting enemy junctions using Scramblers (costs 1 heart)
* defending territory from Clips expansion

Read [MISSION.md](MISSION.md) for a thorough description of the game mechanics.

<p align="center">
  <img src="assets/cvc-reel.gif" alt="CogsGuard reel">
</p>

There are many mission configurations available, with different map sizes, junction layouts, and game rules.

Overall, CogsGuard aims to present rich environments with:

- **Territory control**: Capture and defend junctions to score points each tick
- **Role specialization**: Four roles (Miner, Aligner, Scrambler, Scout) with distinct capabilities and dependencies
- **Dense rewards**: Agents receive reward every tick proportional to territory controlled
- **Partial observability**: Agents have limited visibility of the environment
- **Required multi-agent cooperation**: No single role can succeed alone; Miners need Aligners to capture junctions,
  Aligners need Miners for resources, Scramblers must clear enemy territory for Aligners to advance

# About the tournament

## API Docs
The tournament API is documented at [api.observatory.softmax-research.net/docs](https://api.observatory.softmax-research.net/docs). The interactive
OpenAPI spec describes all public endpoints for seasons, matches, leaderboards, and submissions.


## How seasons work

The ALB leaderboard runs in seasons. Each season has two pools:

- **Qualifying pool**: Where new submissions start. Your policy plays matches against other policies in the pool.
- **Competition pool**: Policies that score above a threshold in qualifying get promoted here.

To see active seasons and their pools:
```bash
cogames seasons
```

## How scoring works

When you submit a policy, it gets queued for matches against other policies in its pool. Our focal metric is VOR (Value Over Replacement), which estimates how much your agent improves team performance compared to a baseline.

VOR is calculated by comparing:

- Replacement mean: The average score when only other pool policies play (no candidate)
- Candidate score: The score when your policy plays

The difference tells us how much value your policy adds to a team. A positive VOR means your policy makes teams better; a negative VOR means teams perform worse with your policy than without it.

You can evaluate VOR locally before submitting:

```bash
cogames pickup --policy <YOUR_POLICY> --pool <POOL_POLICY>
```

## Viewing results

To check your submission status and match results:
```bash
cogames submissions
cogames leaderboard --season beta-teams-small
```

# Command Reference

Most commands are of the form `cogames <command> --mission [MISSION] --policy [POLICY] [OPTIONS]`

To specify a `MISSION`, you can:

- Use a mission name from the registry given by `cogames missions` (e.g. `training_facility_1`).
- Use a path to a mission configuration file (e.g. `path/to/mission.yaml`).
- Alternatively, specify a set of missions with `--mission-set`.

To specify a `POLICY`, use one of two formats:

- **URI format** (for checkpoint bundles):

    - Point directly at a checkpoint bundle (directory or `.zip` containing `policy_spec.json`)
    - Examples: `./train_dir/my_run:v5`, `./train_dir/my_run:v5.zip`, `s3://bucket/path/run:v5.zip`
    - Use `:latest` suffix to auto-resolve the highest version: `./train_dir/checkpoints:latest`

- **Key-value format** (for explicit class + weights):

    - `class=`: Policy shorthand or full class path from `cogames policies`, e.g. `class=lstm` or
    `class=cogames.policy.random.RandomPolicy`.
    - `data=`: Optional path to a weights file (e.g., `weights.safetensors`). Must be a file, not a directory.
    - `proportion=`: Optional positive float specifying the relative share of agents that use this policy (default: 1.0).
    - `kw.<arg>=`: Optional policy `__init__` keyword arguments (all values parsed as strings).

You can view all the commands with
```bash
cogames --help
```
and you can view help for a given command with:
```bash
cogames [COMMAND] --help
```

In [ ]:
# <<hide-input>>
import sys
from collections import defaultdict

# Clear cached modules
for mod in list(sys.modules.keys()):
    if mod.startswith("cogames") or mod.startswith("rich"):
        del sys.modules[mod]

import typer
from click import Context
from IPython.display import Markdown, display

from cogames.main import app


def build_command_groups_from_cli():
    """Build command groups by introspecting the CLI's rich_help_panel attributes."""
    panel_to_commands = defaultdict(list)

    for cmd_info in app.registered_commands:
        if getattr(cmd_info, "hidden", False):
            continue
        panel = getattr(cmd_info, "rich_help_panel", None)
        if panel is None or isinstance(panel, typer.models.DefaultPlaceholder):
            continue
        name = cmd_info.name or (cmd_info.callback.__name__ if cmd_info.callback else None)
        if name:
            panel_to_commands[panel].append(name)

    for group_info in app.registered_groups:
        group_name = group_info.name
        if not group_name or getattr(group_info, "hidden", False):
            continue
        sub_app = group_info.typer_instance
        if sub_app:
            for sub_cmd_info in sub_app.registered_commands:
                if getattr(sub_cmd_info, "hidden", False):
                    continue
                panel = getattr(sub_cmd_info, "rich_help_panel", None)
                if panel is None or isinstance(panel, typer.models.DefaultPlaceholder):
                    continue
                sub_name = sub_cmd_info.name or (sub_cmd_info.callback.__name__ if sub_cmd_info.callback else None)
                if sub_name:
                    panel_to_commands[panel].append(f"{group_name} {sub_name}")

    return dict(panel_to_commands)


def get_click_command(cmd_name: str):
    """Get the Click command object for a command name."""
    click_app = typer.main.get_command(app)
    if " " in cmd_name:
        parent, child = cmd_name.split(" ", 1)
        parent_cmd = click_app.commands.get(parent)
        return parent_cmd.commands.get(child) if parent_cmd and hasattr(parent_cmd, "commands") else None
    return click_app.commands.get(cmd_name)


def display_command_reference():
    """Display command reference with proper heading structure."""
    command_groups = build_command_groups_from_cli()

    for group_name, cmd_names in command_groups.items():
        display(Markdown(f"## {group_name} Commands"))
        for cmd_name in cmd_names:
            display(Markdown(f"### `cogames {cmd_name}`"))
            cmd = get_click_command(cmd_name)
            if cmd:
                ctx = Context(cmd, info_name=f"cogames {cmd_name}")
                print(cmd.get_help(ctx))


display_command_reference()

## Missions Commands

### `cogames missions`

                                                                                                                   
 Usage: cogames missions [OPTIONS] SITE                                                                            
                                                                                                                   

List available missions.                                                                                          
                                                                                                                   
 This command has three modes:                                                                                     
                                                                                                                   
 1. List sites: Run with no arguments to see all available sites.                                                  
                                                                                                                   
 2. List missions at a site: Pass a site name (e.g., 'cogsguard_machina_1') to see its missions.                   
                                                                                                                   
 3. Describe a mission: Use -m to describe a specific mission. Only in this mode do --cogs, --variant, --format,   
 and --save have any effect.

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   site      TEXT  Filter by site (e.g., cogsguard_machina_1)                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─ Hardware ──────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --device                   DEVICE    Device to train on (auto, cpu, cuda, mps) [default: cpu]                   │
│ --num-workers              N [x>=1]  Number of worker processes [default: (CPU cores)]                          │
│ --parallel-envs            N [x>=1]  Number of parallel environments                                            │
│ --vector-batch-size        N [x>=1]  Vectorized environment batch size                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Reproducibility ───────────────────────────────────────────────────────────────────────────────────────────────╮
│ --seed            N [x>=0]  Seed for training RNG [default: 42]                                                 │
│ --map-seed        N [x>=0]  MapGen seed for procedural map layout [default: (same as --seed)]                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --checkpoints        DIR  Path to save training checkpoints [default: ./train_dir]                              │
│ --log-outputs             Log training outputs                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Other ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help  -h        Show this message and exit                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Examples:                                                                                                         
 cogames tutorial train -m cogsguard_machina_1.basic                   Basic training                              
 cogames tutorial train -m cogsguard_machina_1.basic -p class=baseline                                             
 Train baseline policy                                                                                             
 cogames tutorial train -p ./train_dir/my_run:v5                  Continue from checkpoint                         
 cogames tutorial train -p class=lstm,data=./weights.safetensors  Load weights into class                          
 cogames tutorial train -m mission_1 -m mission_2                 Curriculum (rotates)                             
 Wildcard patterns:                                                                                                
 cogames tutorial train -m 'machina_2_bigger:*'                   All missions on machina_2_bigger                 
 cogames tutorial train -m '*:shaped'                             All "shaped" missions                            
 cogames tutorial train -m 'machina*:shaped'                      All "shaped" on machina maps

# Citation

If you use CoGames in your research, please cite:

```bibtex
@software{cogames2025,
  title={CoGames: Multi-Agent Cooperative Game Environments},
  author={Softmax},
  year={2025},
  url={https://github.com/Metta-AI/cogames}
}
```
